# Are Asteroseismic stars aperiodic?

First plot the Periodic stars with asteroseismic stars overlayed, then plot the aperiodic stars.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import astropy.stats as aps
from astropy.io import fits
import aviary as av
from tqdm import trange

from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import gridplot
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform
output_notebook()

Loading BokehJS ...

Load the data sets.

In [2]:
df = pd.read_csv("../../data/bokeh_df.csv")  # Periodic data.
adf = pd.read_csv("../../data/aperiodic.csv")  # Aperiodic
adf = adf.iloc[np.isfinite(adf.Prot.values) & np.isfinite(adf.vb.values) & (adf.Prot_err.values < 8)]

# Load asteroseismic stars.
ast1 = pd.read_csv("../../data/vanSaders.txt")
# a_prot, a_teff, a_age = ast.period.values, ast.teff.values, ast.AMP_age.values

# Load Gaia-Kepler crossmatch.
with fits.open("../../data/kepler_dr2_1arcsec.fits") as data:
    gaia = pd.DataFrame(data[1].data, dtype="float64")
ast = pd.merge(ast1, gaia, how="left", left_on="KIC", right_on="kepid", suffixes=["_vansaders", ""])
print(np.shape(ast))

def abs_g(m, D):
    """
    m - M = 5log(D) - 5
    M = m - 5log(D) + 5
    """
    return m - 5*np.log10(D) + 5

ast["MG"] = abs_g(ast.phot_g_mean_mag.values, (1./ast.parallax.values)*1e3)

/Users/rangus/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (112) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(21, 120)


Define functions.

In [3]:
def calc_dispersion_bins(teffs, prots, mags, vs, teffrange, protrange, magrange):
    """
    This function is for the total McQuillan sample.
    Calculates dispersions in Teff, Prot, M_G bins.
    
    Args:
        teffs (array): The Teff array.
        prots (array): The Prot array.
        mags (array): The M_G array.
        vs (array): The velocity array.
        teffrange (float): The teff bin-size
        protrange (float): The Prot bin-size
        magrange (float): The M_G bin-size
        
    Returns:
        dispersions (array): The velocity dispersion for each star.
        
    """
    dispersions = np.zeros(len(teffs))
    for i in trange(len(teffs)):
        mask = make_mask([teffs[i], prots[i], mags[i]], [teffs, prots, mags], [teffrange, protrange, magrange])
        vbin = vs[mask]
        dispersions[i] = 1.5*aps.median_absolute_deviation(vbin, ignore_nan=True)
    return dispersions


def make_mask(targets, all_stars, ranges):
    """
    Generic function for binning stars in N-D.
    
    Args:
        targets (array): The properties of the target star (e.g. [teff, prot, mag])
        all_stars (list):  list of arrays of properties for each star.
        ranges (list): The bin sizes.
    """
    ndim = len(targets)
    bounds = []
    m = np.ones(len(all_stars[0]), dtype=bool)
    for i in range(ndim):
        upper_bound = targets[i] + ranges[i]/2.
        lower_bound = targets[i] - ranges[i]/2.
        m &= (lower_bound < all_stars[i]) & (all_stars[i] < upper_bound)
    return m

def calc_dispersion_target(target_teff, target_prot, target_mag, teffs, prots, mags, vels, teffrange, protrange, magrange):
    """
    This function is for calculating dispersions of target stars,  binning in teff, prot and mag.
    
    Args:
        target_teff (float): The Teff of the target star.
        target_prot (float): The prot of the target star.
        target_mag (float): The mag of the target star.
        teffs (array): The Teff array.
        prots (array): The Prot array.
        mags (array): The M_G array.
        vs (array): The velocity array.
        teffrange (float): The teff bin-size
        protrange (float): The Prot bin-size
        magrange (float): The M_G bin-size
        
    Returns:
        dispersions (array): The velocity dispersion for each star.
        
    """
    dispersions = np.zeros(len(target_teff))
    for i in trange(len(target_teff)):
        mask = make_mask([target_teff[i], target_prot[i], target_mag[i]], [teffs, prots, mags], [teffrange, protrange, magrange])
        mask2 = np.isfinite(vels[mask])
        if len(vels[mask][mask2]) > 0:
            dispersions[i] = 1.5*aps.median_absolute_deviation(vels[mask][mask2], ignore_nan=True)
    return dispersions

Calculate ages for astero stars.

In [4]:
# Subtract polynomial for MS stars
mc = df.teff.values < 8000
p = np.polyfit(df.teff.values[mc], df.M_G.values[mc], 5)
G = df.M_G.values - np.polyval(p, df.teff)
aG = ast.MG.values - np.polyval(p, ast.teff_vansaders.values)

astero_disp = calc_dispersion_target(ast.teff_vansaders.values, np.log10(ast.period.values), aG,
                                     df.teff, np.log10(df.prot.values), G, df.vz.values,
                                     350, .25, .4)
p_mp, p_mr, p_a = av.get_avr_coefficients()
astero_kin_age = av.v_to_age(astero_disp, p_mr)

100%|██████████| 21/21 [00:00<00:00, 956.51it/s]


Plot the periodic stars.

In [5]:
source = ColumnDataSource(data=dict(teff=df.teff.values[::3], mg=df.M_G.values[::3], prot=df.prot.values[::3],
                                    age=df.kin_age.values[::3]))
# source = ColumnDataSource(data=dict(teff=df.teff.values[::2], mg=df.M_G.values[::2], prot=df.prot.values[::2],
#                                     age=df.kin_age.values[::2]))

sun = ColumnDataSource(data=dict(teff=np.array([5777]), mg=np.array([4.67]), prot=np.array([26]),
                                 age=np.array([4.56])))
astero = ColumnDataSource(data=dict(teff=ast.teff_vansaders.values, mg=ast.MG.values, prot=ast.period.values,
                                    age=ast.AMP_age.values))

TOOLS = "box_select,lasso_select,help,reset"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=700, height=600, x_range=(7500, 3000), y_range=(11, -2))
color_mapper = LinearColorMapper(palette="Plasma256", low=0, high=10)

left.circle(x='teff', y='mg', color=transform('age', color_mapper), size=3, alpha=0.6, source=source)
left.circle(x='teff', y='mg', color=transform('age', color_mapper), line_color="black", size=15, alpha=1, source=sun)
left.square(x='teff', y='mg', color=transform('age', color_mapper), line_color="black", size=10, alpha=1, source=astero)

left.xaxis.axis_label = 'Teff [K]'
left.yaxis.axis_label = 'M_G'

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=700, height=600, y_axis_type="log", x_range=(7500, 3000), y_range=(1, 80))

right.circle(x='teff', y='prot', color=transform('age', color_mapper), size=3, alpha=0.6, source=source)
right.circle(x='teff', y='prot', color=transform('age', color_mapper), line_color="black", size=15, alpha=1, source=sun)
right.square(x='teff', y='prot', color=transform('age', color_mapper), line_color="black", size=10, alpha=1, source=astero)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Age [Gyr]')
right.add_layout(color_bar, 'right')

right.xaxis.axis_label = 'Teff [K]'
right.yaxis.axis_label = 'Prot [days]'

p = gridplot([[left, right]])

# output_file('CMD_TPD_age_astero.html')

show(p)

Calculate ages for aperiodic stars.

In [6]:
# Subtract polynomial for MS stars
mc = df.teff.values < 8000
# p = np.polyfit(adf.color_teffs.values[mc], adf.abs_G.values[mc], 5)
p = np.polyfit(df.teff.values[mc], df.M_G.values[mc], 5)

apG = adf.abs_G.values - np.polyval(p, adf.color_teffs)
astero_G = ast.MG.values - np.polyval(p, ast.teff_vansaders.values)

dispersions_ap = calc_dispersion_bins(adf.color_teffs.values, np.log10(adf.Prot.values), apG, adf.vb.values, 200, .4, .5)
kin_ages_ap = av.v_to_age(dispersions_ap, p_mr)

astero_disp_ap = calc_dispersion_target(ast.teff_vansaders.values, np.log10(ast.period.values), astero_G,
                                        adf.color_teffs, np.log10(adf.Prot.values), apG, adf.vb.values,
                                        350, .25, .4)
astero_kin_age_ap = av.v_to_age(astero_disp_ap, p_mr)

  0%|          | 0/25197 [00:00<?, ?it/s]/Users/rangus/Applications/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
100%|██████████| 21/21 [00:00<00:00, 815.50it/s]


In [ ]:
adf["kin_age"] = kin_ages_ap
ad

Plot the aperiodic stars.

In [55]:
# a_source = ColumnDataSource(data=dict(teff=adf.color_teffs.values[::2], mg=adf.abs_G.values[::2], prot=adf.Prot.values[::2],
#                                       age=kin_ages_ap[::2]))
a_source = ColumnDataSource(data=dict(teff=adf.color_teffs.values[::3], mg=adf.abs_G.values[::3], prot=adf.Prot.values[::3],
                                      age=kin_ages_ap[::3]))

TOOLS = "box_select,lasso_select,help,reset"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=700, height=600, x_range=(7500, 3000), y_range=(11, -2))
color_mapper = LinearColorMapper(palette="Plasma256", low=0, high=10)

left.circle(x='teff', y='mg', color=transform('age', color_mapper), size=3, alpha=0.6, source=a_source)
left.circle(x='teff', y='mg', color=transform('age', color_mapper), line_color="black", size=15, alpha=1, source=sun)
left.square(x='teff', y='mg', color=transform('age', color_mapper), line_color="black", size=10, alpha=1, source=astero)

left.xaxis.axis_label = 'Teff [K]'
left.yaxis.axis_label = 'M_G'

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=700, height=600, y_axis_type="log", x_range=(7500, 3000), y_range=(1, 80))

right.circle(x='teff', y='prot', color=transform('age', color_mapper), size=3, alpha=0.6, source=a_source)
right.circle(x='teff', y='prot', color=transform('age', color_mapper), line_color="black", size=15, alpha=1, source=sun)
right.square(x='teff', y='prot', color=transform('age', color_mapper), line_color="black", size=10, alpha=1, source=astero)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Age [Gyr]')
right.add_layout(color_bar, 'right')

right.xaxis.axis_label = 'Teff [K]'
right.yaxis.axis_label = 'Prot [days]'

p = gridplot([[left, right]])

# output_file('CMD_TPD_aperiodic_asteroseismic.html')

show(p)

In [63]:
TOOLS = "box_select,lasso_select,help,reset"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=700, height=600, x_range=(7500, 3000), y_range=(11, -2))
color_mapper = LinearColorMapper(palette="Plasma256", low=0, high=10)

left.circle(x='teff', y='mg', color=transform('age', color_mapper), size=3, alpha=0.6, source=source)
left.circle(x='teff', y='mg', color=transform('age', color_mapper), size=3, alpha=0.6, source=a_source)
left.circle(x='teff', y='mg', color=transform('age', color_mapper), line_color="black", size=15, alpha=1, source=sun)
left.square(x='teff', y='mg', color=transform('age', color_mapper), line_color="black", size=10, alpha=1, source=astero)

left.xaxis.axis_label = 'Teff [K]'
left.yaxis.axis_label = 'M_G'

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=700, height=600, y_axis_type="log", x_range=(7500, 3000), y_range=(1, 80))

right.circle(x='teff', y='prot', color=transform('age', color_mapper), size=3, alpha=0.6, source=source)
right.circle(x='teff', y='prot', color=transform('age', color_mapper), size=3, alpha=0.6, source=a_source)
right.circle(x='teff', y='prot', color=transform('age', color_mapper), line_color="black", size=15, alpha=1, source=sun)
right.square(x='teff', y='prot', color=transform('age', color_mapper), line_color="black", size=10, alpha=1, source=astero)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Age [Gyr]')
right.add_layout(color_bar, 'right')

right.xaxis.axis_label = 'Teff [K]'
right.yaxis.axis_label = 'Prot [days]'

p = gridplot([[left, right]])

output_file('combo0.html')

show(p)